In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# show all outputs of a cell (such as if df.head() and df.tail() are in the same cell)
#default is 'last_expr'
from pandasql import sqldf
sql = lambda q: sqldf(q, globals()) 
import pandas as pd

classes=[('Bismarck','bb','Germany',8,15,42000),
('Iowa','bb','USA',9,16,46000),('Kongo','bc','Japan',8,14,32000),
('North Carolina','bb','USA',12,16,37000),('Renown','bc','Gt.Britain',6,15,32000),
('Revenge','bb','Gt.Britain',8,15,29000),('Tennessee','bb','USA',12,14,32000),
('Yamato','bb','Japan',9,18,65000)]

battles=[('Guadalcanal','1942-11-15 00:00:00'),('North Atlantic','1941-05-25 00:00:00'),
('North Cape','1943-12-26 00:00:00'),('Surigao Strait','1944-10-25 00:00:00'),
('#Cuba62a'   , '1962-10-20'),('#Cuba62b'   , '1962-10-25')]

ships=[('California','Tennessee',1921),('Haruna','Kongo',1916),
('Hiei','Kongo',1914),('Iowa','Iowa',1943),('Kirishima','Kongo',1915),
('Kongo','Kongo',1913),('Missouri','Iowa',1944),('Musashi','Yamato',1942),
('New Jersey','Iowa',1943),('North Carolina','North Carolina',1941),
('Ramillies','Revenge',1917),('Renown','Renown',1916),('Repulse','Renown',1916),
('Resolution','Renown',1916),('Revenge','Revenge',1916),('Royal Oak','Revenge',1916),
('Royal Sovereign','Revenge',1916),('Tennessee','Tennessee',1920),
('Washington','North Carolina',1941),('Wisconsin','Iowa',1944),('Yamato','Yamato',1941),
('South Dakota','North Carolina',1941)]

outcomes=[('Bismarck','North Atlantic','sunk'),
('California','Surigao Strait','OK'),('Duke of York','North Cape','OK'),
('Fuso','Surigao Strait','sunk'),('Hood','North Atlantic','sunk'),
('King George V','North Atlantic','OK'),('Kirishima','Guadalcanal','sunk'),
('Prince of Wales','North Atlantic','damaged'),('Rodney','North Atlantic','OK'),
('Schamhorst','North Cape','sunk'),('South Dakota','Guadalcanal','damaged'),
('Tennessee','Surigao Strait','OK'),('Washington','Guadalcanal','OK'),
('West Virginia','Surigao Strait','OK'),('Yamashiro','Surigao Strait','sunk'),
('California','Guadalcanal','damaged')]

classes=pd.DataFrame(classes,columns=['class','type','country','numGuns','bore','displacement',])
battles=pd.DataFrame(battles,columns=['name','date'])
ships=pd.DataFrame(ships,columns=['name','class','launched'])
outcomes=pd.DataFrame(outcomes,columns=['ship','battle','result'])
Ships = ships
Classes=classes
Outcomes = outcomes
Battles=battles

In [39]:
'''
The database of naval ships that took part in World War II is under consideration. 

A class is normally assigned either the name of the first ship built according to the 
corresponding design, or a name that is different from any ship name in the database. 
The ship whose name is assigned to a class is called a lead ship.

The Classes relation includes the name of the class, type 
(can be either bb for a battle ship, or bc for a battle cruiser), 
country the ship was built in, the number of main guns, gun caliber (bore diameter in inches), 
and displacement (weight in tons). 

The Ships relation holds information about the ship name, 
the name of its corresponding class, and the year the ship was launched. 

The Battles relation contains names and dates of battles the ships participated in, 
and the Outcomes relation - the battle result for a given ship (may be sunk, damaged, or OK, 
the last value meaning the ship survived the battle unharmed). 

Notes: 1) The Outcomes relation may contain ships not present in the Ships relation. 
2) A ship sunk can’t participate in later battles. 3) For historical reasons, 
lead ships are referred to as head ships in many exercises.4) 
A ship found in the Outcomes table but not in the Ships table is still 
considered in the database. This is true even if it is sunk.

The database consists of the following relations: 
Classes(class, type, country, numGuns, bore, displacement) 
Ships(name, class, launched) 
Battles(name, date) 
Outcomes(ship, battle, result) 
Ships in classes all have the same general design.

Observation: there are many ships in the outcomes table that are not lead ships, so how would we know their class?
(A lead ship's name is its class name).
We would join the outcomes ship name to the ships table's ship's name, and inspect the class field in this table
If the outcomes ship is not in the ships table, then there's no way to link it to the class table.

''';

In [2]:
q='''
Select * from ships
LIMIT 1
'''
sql(q)

,name,class,launched
0,California,Tennessee,1921


**Exercise: 57 (Serge I: 2003-02-14)** <br>
For classes having irreparable combat losses and at least three ships in the database, display the name of the class and the number of ships sunk.

class, # of ships sunk
where class has at least one ship with irreperable combat losses & (# of ships)>3

unique ships with sunk ships


In [7]:
q='''
WITH out AS
(
 SELECT DISTINCT ship FROM outcomes WHERE result = 'sunk'
),
 t1 AS (SELECT DISTINCT c.class,


(SELECT DISTINCT COUNT(s.name) FROM ships s WHERE s.class = c.class)
+
(SELECT DISTINCT COUNT(o.ship) FROM out o WHERE o.ship = c.class AND 
o.ship NOT IN (SELECT name FROM ships)) ships_qty,
(SELECT DISTINCT COUNT(o.ship)







FROM out o
WHERE o.ship = c.class OR
 (SELECT COUNT(s.name) FROM ships s WHERE s.name = o.ship AND s.class = c.class) > 0
) sunk
FROM classes c
)

SELECT class, sunk FROM t1
WHERE t1.ships_qty >= 3
AND t1.sunk >= 1
'''
sql(q)

,class,sunk
0,Kongo,1


In [4]:
q='''

select classes.class, case when outcomes.result = 'sunk' then 1 else 0 end as result from classes 
left join ships on ships.class = classes.class
left join outcomes on outcomes.ship = ships.name
union
select classes.class, case when outcomes.result = 'sunk' then 1 else 0 end  as result from classes
left join outcomes on outcomes.ship = classes.class


'''
sql(q)

,class,result
0,Bismarck,0
1,Bismarck,1
2,Iowa,0
3,Kongo,0
4,Kongo,1
5,North Carolina,0
6,Renown,0
7,Revenge,0
8,Tennessee,0
9,Yamato,0


In [ ]:
THIS RESULT IS WRONG ON CHECKING.  REVIEW FAQ.

In [24]:
q='''
with temp1 as (
select classes.class, 
case when outcomes.result = 'sunk' then 1 else 0 end as result from classes 
left join ships on ships.class = classes.class
left join outcomes on outcomes.ship = ships.name
union
select classes.class, 
case when outcomes.result = 'sunk' then 1 else 0 end as result from classes
left join outcomes on outcomes.ship = classes.class
),

temp2 as (
select classes.class from classes 
join ships on ships.class = classes.class
where classes.class in
(select classes.class from classes join ships on ships.class = classes.class
group by classes.class
having count(ships.name)>2)
union
select classes.class from classes 
join outcomes on outcomes.ship = classes.class
where classes.class in
(select classes.class from classes join outcomes on outcomes.ship = classes.class
group by classes.class
having count(outcomes.ship)>2)
)

select temp1.class, sum(temp1.result) as sunken  from temp1 
join temp2 on temp2.class = temp1.class
group by temp1.class
having sum(temp1.result) =1
'''
sql(q)

,class,sunken
0,Kongo,1


In [26]:
q='''
with temp2 as (
select classes.class from classes 
join ships on ships.class = classes.class
where classes.class in
(select classes.class from classes join ships on ships.class = classes.class
group by classes.class
having count(ships.name)>2)
union
select classes.class from classes 
join outcomes on outcomes.ship = classes.class
where classes.class in
(select classes.class from classes join outcomes on outcomes.ship = classes.class
group by classes.class
having count(outcomes.ship)>2)
)
select * from temp2
'''
sql(q)

,class
0,Iowa
1,Kongo
2,North Carolina
3,Renown
4,Revenge


In [30]:
q='''
select * from classes join ships on ships.class = classes.class 
(
select class from classes 
where class in
(select classes.class from classes join ships on ships.class=classes.class
join outcomes on outcomes.ship=ships.name where result='sunk')
and class in 
(select classes.class from classes 
join ships on ships.class = classes.class
group by classes.class
having count(ships.name)>2)
)
'''
sql(q)

,class
0,Kongo


**Exercise: 56 (Serge I: 2003-02-16)** <br>
For each class, find out the number of ships of this class that were sunk in battles.  <br>
Result set: class, number of ships sunk.

In [20]:
#to have classes with a count of zero, you'd need to start from the class table

q='''
with temp as (
select classes.class, outcomes.ship, outcomes.result,
case when result = 'sunk' then 1 else 0 end as 'sum'
from classes
left join ships on ships.class = classes.class
left join outcomes on outcomes.ship = ships.name
union
select classes.class, outcomes.ship, outcomes.result,
case when result = 'sunk' then 1 else 0 end as 'sum'
from classes
left join outcomes on outcomes.ship = classes.class
)
select class, sum(sum) as result
from temp
group by class
'''
sql(q)

,class,result
0,Bismarck,1
1,Iowa,0
2,Kongo,1
3,North Carolina,0
4,Renown,0
5,Revenge,0
6,Tennessee,0
7,Yamato,0


**Exercise: 55 (Serge I: 2003-02-16)** <br>
For each class, determine the year the first ship of this class was launched. 
If the lead ship’s year of launch is not known, get the minimum year of launch for the ships of this class. <br>
Result set: class, year.

In [28]:
q='''
select classes.class, min(launched)
from classes 
left join outcomes on outcomes.ship = classes.class
left join ships on ships.class = classes.class
group by classes.class

'''
sql(q)

,class,min(launched)
0,Bismarck,NaN
1,Iowa,1943.0
2,Kongo,1913.0
3,North Carolina,1941.0
4,Renown,1916.0
5,Revenge,1916.0
6,Tennessee,1920.0
7,Yamato,1941.0


**Exercise: 54 (Serge I: 2003-02-14)<br>
With a precision of two decimal places, determine the average number of guns for all<br> battleships (including the ones in the Outcomes table).**

In [11]:
q='''
with temp as (
select name, numGuns from classes join ships on ships.class = classes.class where type='bb'
union
select ship, numGuns from classes join outcomes on outcomes.ship = classes.class where 
type = 'bb'
)
select cast(avg(numGuns+0.0) as decimal(10,2)) from temp

'''
sql(q)

,"cast(avg(numGuns+0.0) as decimal(10,2))"
0,9.625


**Exercise: 53 (Serge I: 2002-11-05) <br>
With a precision of two decimal places, determine the average number of guns for the <br> battleship classes.**

In [3]:
#an interesting aspect of arithmetic arguments in sql is that the result is always
#the data type of the arguments.  If average() is on integers, even if the answer SHOULD have
# a fractional amount, this amount is discarded!  

#Solution turn numGuns into a float by adding 0.0!!!!
#10 is the maximum length, and 2 is the number of decimal places
q='''
select cast(avg(numGuns+0.0) as decimal(10,2)) from classes 
where type ='bb'
'''
sql(q)

,"cast(avg(numGuns+0.0) as decimal(3,2))"
0,9.666667


**Exercise: 52 (qwrqwr: 2010-04-23) <br>
Determine the names of all ships in the Ships table that can be a Japanese battleship having at least nine main guns with a caliber of less than 19 inches and a displacement of not more than 65 000 tons.**
<br>

In [154]:
q='''
select name from ships join classes on classes.class = ships.class
where (type = 'bb' or type IS NULL)
and (country = 'Japan' or country IS NULL)
and (numGuns>=9 or numGuns IS NULL)
and (bore <19 or bore IS NULL)
and (displacement <=65000 or displacement IS NULL)
'''
sql(q)

,name
0,Musashi
1,Yamato


** Exercise: 51 (Serge I: 2003-02-17)<br>
Find the names of the ships with the largest number of guns among all ships having the same displacement (including ships in the Outcomes table).**

In [131]:
q='''

SELECT name
FROM 
(SELECT O.ship AS name, numGuns, displacement
 FROM Outcomes O INNER JOIN 
 Classes C ON O.ship = C.class AND 
 O.ship NOT IN (SELECT name FROM Ships) 
 UNION
 SELECT S.name AS name, numGuns, displacement 
 FROM Ships S INNER JOIN 
 Classes C ON S.class = C.class ) OS  
 
 INNER JOIN 
 (SELECT MAX(MaxNumGuns) AS Max, displacement from
 (SELECT MAX(numGuns) AS MaxNumGuns, displacement
 
 FROM Outcomes O INNER JOIN 
 Classes C ON O.ship = C.class AND 
 O.ship NOT IN (SELECT name FROM Ships) 
 GROUP BY displacement
 UNION
 SELECT MAX(numGuns) AS MaxNumGuns, displacement
 FROM Ships S INNER JOIN 
 Classes C ON S.class = C.class
 GROUP BY displacement) GD 
 
 GROUP BY displacement
 ) GD2
 
 ON OS.numGuns = GD2.Max AND 
 OS.displacement = GD2.displacement;
 
 '''
sql(q)

,name
0,Bismarck
1,California
2,Iowa
3,Missouri
4,Musashi
5,New Jersey
6,North Carolina
7,Ramillies
8,Revenge
9,Royal Oak


In [132]:
q='''
Creating multiple cte's and joining them; notice there is no WITH on the second CTE 
WITH SomeCTE AS (
  SELECT ...
  FROM ...
), AnotherCTE AS (
  SELECT ...
  FROM ...
)
SELECT *
FROM SomeCTE LEFT JOIN
     AnotherCTE ON ...
;
'''

In [127]:
q='''
with temp as (
select name, classes.class, numGuns, displacement from ships 
join classes on classes.class = ships.class
union 
select ship, classes.class, numGuns, displacement from outcomes 
join classes on classes.class = outcomes.ship
),


temp2 as (
select displacement, max(table_max) as max from
(select displacement, max(numGuns) as table_max from classes 
join ships on ships.class = classes.class
group by displacement
union
select displacement, max(numGuns) from classes 
join outcomes on outcomes.ship = classes.class
group by displacement) temp
group by displacement)


select name from temp
join temp2 on (temp2.displacement = temp.displacement)
and (temp2.max = temp.numGuns)


'''
sql(q)

,name
0,Bismarck
1,California
2,Iowa
3,Missouri
4,Musashi
5,New Jersey
6,North Carolina
7,Ramillies
8,Revenge
9,Royal Oak


**Exercise: 50 (Serge I: 2002-11-05) <br>
Find the battles in which Kongo-class ships from the Ships table were engaged.**

In [48]:
q='''
select battle from outcomes join ships on ships.name = outcomes.ship
where ships.class = 'Kongo'
group by battle

'''
sql(q)

,battle
0,Guadalcanal


**Exercise: 49 (Serge I: 2003-02-17)<br>
Find the names of the ships having a gun caliber of 16 inches (including ships in the Outcomes table).**

In [41]:
q='''
select name from classes join ships on ships.class = classes.class 
where bore =16
union 
select ship from classes join outcomes on outcomes.ship = classes.class
where bore =16
'''
sql(q)

,name
0,Iowa
1,Missouri
2,New Jersey
3,North Carolina
4,South Dakota
5,Washington
6,Wisconsin


**Exercise: 48 (Serge I: 2003-02-16) <br>
Find the ship classes having at least one ship sunk in battles.**

In [21]:
q='''
with temp as (select classes.class  from classes join ships on ships.class = classes.class
left join outcomes on outcomes.ship = ships.name
where result ='sunk'
group by classes.class having count(outcomes.result)>=1

union

select classes.class from classes join outcomes on outcomes.ship = classes.class
where result = 'sunk'
group by classes.class having count(outcomes.result)>=1
)
select * from temp

'''
sql(q)

,class
0,Bismarck
1,Kongo


**Exercise: 57 (Serge I: 2003-02-14) <br>
For classes having irreparable combat losses and at least three ships in the database, display the name of the class and the number of ships sunk.**


join classes and outcomes and only include classes where result = 'sunk' for at least one ship
only include classes with >= 3 ships

class records with ship information for ships with classes
the above with outcomes information
some outcomes ships are not in ships table (those whose names are a class, so
join on 

In [29]:
q='''
select c.class, count(*) from classes c 
left join ships s on s.class=c.class
left join outcomes o on o.ship=c.class
group by c.class
having count(*) >=3

'''
sql(q)

,class,count(*)
0,Iowa,4
1,Kongo,4
2,North Carolina,3
3,Renown,3
4,Revenge,4


In [15]:
q='''
select * from outcomes
LIMIT 3
'''
sql(q)

,ship,battle,result
0,Bismarck,North Atlantic,sunk
1,California,Surigao Strait,OK
2,Duke of York,North Cape,OK


In [6]:
q='''
select * from classes
LIMIT 3
'''
sql(q)

,class,type,country,numGuns,bore,displacement
0,Bismarck,bb,Germany,8,15,42000
1,Iowa,bb,USA,9,16,46000
2,Kongo,bc,Japan,8,14,32000


**Exercise: 46 (Serge I: 2003-02-14) <br>
For each ship that participated in the Battle of Guadalcanal, get its name, displacement, and the number of guns.**

join ship columns to outcomes ship table
join classes columns to combined table where class values match OR where 
class matches outcomes ship name

some outcomes ships are not in ships table, so therefore there wont be a ships table class
but these ships if lead ships will have the same value as the class value so they'll be
in classes table

In [2]:
q='''

select ship, displacement, numGuns from outcomes
left join ships on ships.name=outcomes.ship
left join classes on classes.class=ships.class or classes.class=outcomes.ship
where battle='Guadalcanal'

'''
sql(q)

,ship,displacement,numGuns
0,Kirishima,32000,8
1,South Dakota,37000,12
2,Washington,37000,12
3,California,32000,12


In [13]:
q='''
select ship, displacement, numGuns from 
outcomes, ships, classes
where outcomes.ship = ships.name
and ships.class = classes.class
and battle ='Guadalcanal'
'''
sql(q)

,ship,displacement,numGuns
0,Kirishima,32000,8
1,South Dakota,37000,12
2,Washington,37000,12
3,California,32000,12


In [15]:
q='''
select * from outcomes
limit 2
'''
sql(q)

,ship,battle,result
0,Bismarck,North Atlantic,sunk
1,California,Surigao Strait,OK


In [3]:
q='''
select distinct ship, displacement, numguns
from classes left join ships
on classes.class=ships.class

where battle='Guadalcanal'
'''


In [4]:
q='''
select distinct ship, displacement, numguns
from Classes left join Ships
on classes.class=ships.class
right join Outcomes
on Classes.class=ship
or ships.name=ship
where battle='Guadalcanal';
'''


In [30]:
select distinct ship, displacement, numguns
from Classes left join Ships
on classes.class=ships.class
right join Outcomes
on Classes.class=ship
or ships.name=ship
where battle='Guadalcanal';

SyntaxError: invalid syntax (<ipython-input-30-8b20afd84e45>, line 1)

In [5]:
q='''
select name, displacement, numGuns 
from classes left join ships on ships.class = classes.class
right join outcomes on outcomes.ship=ships.name



'''


In [6]:
q='''
select outcomes.ship, classes.displacement, classes.numGuns from outcomes
join ships on outcomes.ship = ships.name
join ships on ships.class = classes.class
where battle ='Guadalcanal'

'''


**Exercise: 45 (Serge I: 2002-12-04) <br>
Find all ship names consisting of three or more words (e.g., King George V).
Consider the words in ship names to be separated by single spaces, and the ship names to have no leading or trailing spaces.**

In [27]:
# '%_ _% _%'  the underscore means we need at least one letter between each space.  
# without the _, we could have 0 characters between each space, since % can be 0
q='''
select RTRIM(LTRIM(name)) as name from ships
where name like '%_ _% _%' 
union
select RTRIM(LTRIM(ship)) as name from outcomes
where ship like '%_ _% _%'

'''
sql(q)

,name
0,Duke of York
1,King George V
2,Prince of Wales


**Exercise: 44 (Serge I: 2002-12-04) <br>
Find all ship names beginning with the letter R.**

In [31]:
q='''
select name from ships
where name like 'R%'
union
select ship from outcomes
where ship like 'R%'
'''
sql(q)

,name
0,Ramillies
1,Renown
2,Repulse
3,Resolution
4,Revenge
5,Rodney
6,Royal Oak
7,Royal Sovereign


**Exercise: 43 (qwrqwr: 2011-10-28) <br>
Get the battles that occurred in years when no ships were launched into water.**

In [42]:
#http://sqlite.org/lang_datefunc.html
#in mysql, just use year(date)
q='''
with battle_years as (
select name, strftime('%Y', date) as year from battles
)
select battle_years.name from battle_years
left join ships on ships.launched=battle_years.year
where launched is null
'''
sql(q)

,battle_years.name
0,#Cuba62a
1,#Cuba62b


In [12]:
q='''
with battle_years as (
select name, strftime('%Y', date) as year from battles
)
select battle_years.name from battle_years
where year not in (select launched from ships where launched is not null)
'''
sql(q)

,battle_years.name
0,#Cuba62a
1,#Cuba62b


** Exercise: 42 (Serge I: 2002-11-05) <br>
Find the names of ships sunk at battles, along with the names of the corresponding battles. **

In [7]:
q='''
select ship,battle from outcomes where result = 'sunk'
'''
sql(q)

,ship,battle
0,Bismarck,North Atlantic
1,Fuso,Surigao Strait
2,Hood,North Atlantic
3,Kirishima,Guadalcanal
4,Schamhorst,North Cape
5,Yamashiro,Surigao Strait


**Exercise: 40 (Serge I: 2002-11-05) <br>
For the ships in the Ships table that have at least 10 guns, get the class, name, and country.**


In [14]:
q='''
select ships.name, ships.class, classes.country from ships 
join classes on classes.class = ships.class
where numGuns>=10
'''
sql(q)

,name,class,country
0,North Carolina,North Carolina,USA
1,South Dakota,North Carolina,USA
2,Washington,North Carolina,USA
3,California,Tennessee,USA
4,Tennessee,Tennessee,USA


**Exercise: 39 (Serge I: 2003-02-14) <br>
Find the ships that "survived for future battles"; that is, after being damaged in a battle, they participated in another one, which occurred later.** 

We need to get ships from the outcomes table such that:
ship had a battle that damaged it on an earlier date
ship had a battle on a later date

select ships that were damaged on a date
and then, select these same ships but only those with dates>first date

In [33]:
q='''
select distinct o2.ship from
(select * from outcomes o join battles on battles.name = o.battle where result = 'damaged') as o1
join (select * from outcomes o join battles on battles.name = o.battle ) as o2
on o1.ship=o2.ship 
where o2.date > o1.date

'''
sql(q)

,o2.ship
0,California


In [37]:
q='''
select distinct o2.ship from
(select * from outcomes o join battles on battles.name = o.battle where result = 'damaged') as o1, 
(select * from outcomes o join battles on battles.name = o.battle ) as o2
where o1.ship = o2.ship 
and   o2.date > o1.date
'''
sql(q)

,o2.ship
0,California


**Exercise: 38 (Serge I: 2003-02-19) <br>
Find countries that ever had classes of both battleships (‘bb’) and cruisers (‘bc’).**

In [4]:
#place except where intersect is so that rows of first dataset are returned except for rows of first dataset
#that are also in the second dataset
q='''
select country from classes
where type='bb'
group by country
intersect 
select country from classes
where type='bc'
group by country

'''
sql(q)

,country
0,Gt.Britain
1,Japan


**Exercise: 37 (Serge I: 2003-02-17) <br>
Find classes for which only one ship exists in the database (including the Outcomes table).**

In [ ]:
#can't seem to use table aliases in select statement in unioned tables
# you can't use aliases in the where clause period, anyway

In [30]:
# union does not include duplicate rows, while union all does include them.
#duplicates from outcomes table won't be included
q='''
with temp as (
select class, name from ships
union
select ship, ship from outcomes 
where ship in (select class from classes) 
) 
select class from temp
group by class
having count(class)=1
 
'''
sql(q)

,class
0,Bismarck


In [32]:
q='''
select s.class from classes c inner join (
select name, class from ships
Union
select o.ship, c2.class from classes c2 inner join outcomes o on c2.class=o.ship 
) s on c.class=s.class
group by s.class having count(s.class)=1
'''
sql(q)

,class
0,Bismarck


**Exercise: 36 (Serge I: 2003-02-17) <br>
List the names of lead ships in the database (including the Outcomes table).**

In [30]:
q='''
select name from ships join classes on classes.class = ships.name
union
select ship from outcomes join classes on classes.class = outcomes.ship
'''
sql(q)

,name
0,Bismarck
1,Iowa
2,Kongo
3,North Carolina
4,Renown
5,Revenge
6,Tennessee
7,Yamato


**Exercise: 34 (Serge I: 2002-11-04)<br>
In accordance with the Washington Naval Treaty concluded in the beginning of 1922, it was prohibited to build battle ships with a displacement of more than 35 thousand tons. 
Get the ships violating this treaty (only consider ships for which the year of launch is known). 
List the names of the ships.**


In [12]:
q='''
select name from ships s join classes c on c.class = s.class 
where c.type = 'bb' and c.displacement >35000 and launched >=1922
'''
sql(q)

,name
0,Iowa
1,Missouri
2,New Jersey
3,Wisconsin
4,North Carolina
5,South Dakota
6,Washington
7,Musashi
8,Yamato


**Exercise: 33 (Serge I: 2002-11-02) <br>
Get the ships sunk in the North Atlantic battle. <br>
Result set: ship.**

In [3]:
q='''
select ship from outcomes where battle = 'North Atlantic' and result = 'sunk'
'''
sql(q)

,ship
0,Bismarck
1,Hood


**Exercise: 32 (Serge I: 2003-02-17) <br>
One of the characteristics of a ship is one-half the cube of the calibre of its main guns (mw). <br>
Determine the average ship mw with an accuracy of two decimal places for each country having ships in the database.**

select country, name ship_name, round(avg((.5*bore*bore*bore)),2) as mw from classes
left join ships on ships.class=classes.class 
group by country, ship_name
union

(classes 'class' = ships 'class') and (outcomes 'ship' = ships 'name')
where the name of a ship is sometimes the name of a class, so we can include all outcome ships that were the first of their kind built, since their name = class name.  We don't want to include outcome ships whose name=class is already in the ships table though, or else we'd be double counting that ship.

In [28]:
#sqlite does not have the function power(var,3), which raises var to the 3rd power
#sqlite also must use round() instead of cast(var as decimal(10,2))
q='''

with temp as (
select country, ships.name, bore from classes c join ships on ships.class=c.class
union 
select country, ship, bore from classes c join outcomes o on o.ship=c.class
and o.ship not in (select name from ships)    )
select country, round(avg(.5*(bore*bore*bore)),2) from temp
group by country
'''
sql(q)

,country,"round(avg(.5*(bore*bore*bore)),2)"
0,Germany,1687.50
1,Gt.Britain,1687.50
2,Japan,1886.67
3,USA,1897.78


In [32]:
q='''
select country, c.class, bore from classes c join ships on ships.class=c.class
union 
select country, ship, bore from classes c join outcomes o on o.ship=c.class
and o.ship not in (select name from ships)  
'''
sql(q)

,country,class,bore
0,Germany,Bismarck,15
1,Gt.Britain,Renown,15
2,Gt.Britain,Revenge,15
3,Japan,Kongo,14
4,Japan,Yamato,18
5,USA,Iowa,16
6,USA,North Carolina,16
7,USA,Tennessee,14


In [31]:
q='''
select country, ships.name, bore from classes c join ships on ships.class=c.class
union 
select country, ship, bore from classes c join outcomes o on o.ship=c.class
and o.ship not in (select name from ships)  
'''
sql(q)

,country,name,bore
0,Germany,Bismarck,15
1,Gt.Britain,Ramillies,15
2,Gt.Britain,Renown,15
3,Gt.Britain,Repulse,15
4,Gt.Britain,Resolution,15
5,Gt.Britain,Revenge,15
6,Gt.Britain,Royal Oak,15
7,Gt.Britain,Royal Sovereign,15
8,Japan,Haruna,14
9,Japan,Hiei,14


**Exercise: 31 (Serge I: 2002-10-22)<br>
For ship classes with a gun caliber of 16 in. or more, display the class and the country.**

In [43]:
q='''
select class, country from classes
where bore>=16
group by class, country
'''
sql(q)

,class,country
0,Iowa,USA
1,North Carolina,USA
2,Yamato,Japan
